In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy
import pandas
import matplotlib
import seaborn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

df = pandas.read_csv(r'/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
#Data Cleaning (NaN Values)

print(df.isna().sum())


#no sample has empty values

In [ ]:
#structure and statistics information

print(df.info(verbose=True))
print("Nº Linhas {0}\nNº Colunas {1}".format(df.shape[0], df.shape[1]))
print("Features: {0}\n".format(df.columns.tolist()))

#dataframe compose only by numeric data

print(df.describe(include = "all"))

In [ ]:
#Probability distribution



fig,axes=matplotlib.pyplot.subplots(3,3, figsize = (15,10))
seaborn.histplot (data = df,x="Pregnancies",kde=True, ax=axes[0,0])
seaborn.histplot (data = df,x="Glucose",kde=True,ax=axes[0,1])
seaborn.histplot (data = df,x="BloodPressure",kde=True, ax=axes[0,2])
seaborn.histplot (data = df,x="SkinThickness",kde=True,ax=axes[1,0])
seaborn.histplot (data = df,x="Insulin",kde=True, ax=axes[1,1])
seaborn.histplot (data = df,x="BMI",kde=True,ax=axes[1,2])
seaborn.histplot (data = df,x="DiabetesPedigreeFunction",kde=True, ax=axes[2,0])
seaborn.histplot (data = df,x="Age",kde=True,ax=axes[2,1])

#matplotlib.axes.Axes.set_xlim(self=bar_p,left=0, right=20)
#matplotlib.axes.Axes.set_ylim(self=bar_p,bottom=0, top=5)
matplotlib.pyplot.show()

In [ ]:
corr_data = df.corr()
corr_data.style.background_gradient(cmap='coolwarm', axis=None).set_precision(6)

#most features do not present a linear relation between them

In [ ]:


fig,axes=matplotlib.pyplot.subplots(1,3, figsize = (15,8))
seaborn.kdeplot (data = df,x="Glucose",y="BMI",  ax=axes[0])
seaborn.kdeplot (data = df,x="Glucose",y="Age",ax=axes[1])
seaborn.kdeplot (data = df,x="Glucose",y="Insulin", ax=axes[2])


matplotlib.pyplot.show()

In [ ]:
Y = df ["Outcome"]
X = df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)


model = LogisticRegression(max_iter=300)
model.fit(X_train,Y_train)

model_coeff = model.coef_


seaborn.barplot(x=[0, 1, 2, 3, 4, 5, 6, 7], y=model_coeff[0], palette=['blue' if (x < 0.5) else 'red' for x in model_coeff[0]])

print("Model Classes: {0}".format(model.classes_))
i=0
for x in X.columns.tolist():
    print("{2}. {0}: {1}".format(x, model_coeff[0][i], i))
    i+=1


#the feature that most affects the outcome is the DiabetesPedigreeFunction
#feature like Insulin and SkinThickness can me removed to increse model speed

In [ ]:
#Model Evaluation

print(metrics.classification_report (Y_test, model.predict(X_test)))
print("Coefficient of Determination R2: {0}".format(metrics.r2_score(Y_test, model.predict(X_test))))

#the percentage of the total variation that can be explained by the linear relationship 
#between the features and the outcome is 1%
#as we saw, almost no linear relation is presented


print("MS Error: {0}".format(metrics.mean_squared_error(Y_test, model.predict(X_test))))

In [ ]:
metrics.plot_confusion_matrix (model, X_test, Y_test)
matplotlib.pyplot.show()

#9 elements with Type I error
#25 elements with Type II error
#the model made almost triple the mistakes when predicting false negatives
#the model has problems when predicting True outcomes (outcomes==1)

#HOWEVER we can see from the classification_report that there around 1.6 times more samples
#with Outcome==0 than with Outcome==1